# 06. parsnip 패키지

In [2]:
library(tidyverse)
library(tidymodels)
ggplot2::theme_set(theme_bw())
tidymodels_prefer()

In [19]:
set.seed(20221010)
ames_split <- initial_split(ames, prop = 0.8, strata = Sale_Price)
ames_train <- training(ames_split)
ames_test <- testing(ames_split)

# 
# 
### 모델 생성

#### $ y_i = \beta_0 + \beta_1 x_{1i}  + ... + \beta_p x_{pi} $

# 
# 
# 

- 일반 선형 회귀는 최소 제곱법을 사용하여 모델 매개변수 계산

``` 
model <- lm(formula, data, ...) 
```
# 
# 

- 정규화 선형 회귀는 계수를 0으로 축소하여 단순성을 장려하기 위해 최소 제곱 방법에 페널티를 추가. 이것은 베이지안 또는 비 베이지안 기술을 사용하여 실행가능

#### 베이지안 모델

``` 
library(rstanarm)
model <- stan_glm(formula, data, family = "gaussian", ...)
```

# 
#### 비 베이지안 모델
```
library(glm)
model <- glmnet(x = matrix, y = vector, family = "gaussian", ...)
```

# 
# 
# 
### TIDYMODEL을 통한 모델 지정
1. 수학적 구조 (예 : 선형 구조, 랜덤 포레스트, KNN 등)을 기반으로 모델 유형 지정
2. 모델을 적합할 엔진 지정 (**대부분의 경우 Stan또는 glmnet과 같이 사용해야 하는 소프트웨어 패키지를 반영**, 이것들은 그 자체로 모델이며, parsnip은 이를 모델링 엔진으로 사용하여 일관된 인터페이스를 제공)
3. 필요한 경우 모델의 모드를 선언 (회귀 / 분류). 모드는 예측 결과의 유형을 반영. 수치 결과의 경우 

# 
# 
- 회귀 => 일반 선형 회귀

In [9]:
linear_reg() %>% set_engine("lm")

Linear Regression Model Specification (regression)

Computational engine: lm 


- 회귀 => 정규화 회귀(glmnet)

In [10]:
linear_reg() %>% set_engine("glmnet")

Linear Regression Model Specification (regression)

Computational engine: glmnet 


- 회귀 => 정규화 회귀 (rstanarm)

In [11]:
linear_reg() %>% set_engine("stan")

Linear Regression Model Specification (regression)

Computational engine: stan 


# 
# 
### ```parsnip::translate()``` : 사용자 코드를 패키지 구문으로 변환하는 방법에 대한 세부 정보 제공

In [12]:
linear_reg() %>% set_engine("lm") %>% translate()

Linear Regression Model Specification (regression)

Computational engine: lm 

Model fit template:
stats::lm(formula = missing_arg(), data = missing_arg(), weights = missing_arg())

- 회귀 => 정규화 회귀(glmnet)

In [14]:
linear_reg(penalty = 1) %>% set_engine("glmnet") %>% translate()

Linear Regression Model Specification (regression)

Main Arguments:
  penalty = 1

Computational engine: glmnet 

Model fit template:
glmnet::glmnet(x = missing_arg(), y = missing_arg(), weights = missing_arg(), 
    family = "gaussian")

- 회귀 => 정규화 회귀 (rstanarm)

In [15]:
linear_reg() %>% set_engine("stan") %>% translate()

Linear Regression Model Specification (regression)

Computational engine: stan 

Model fit template:
rstanarm::stan_glm(formula = missing_arg(), data = missing_arg(), 
    weights = missing_arg(), family = stats::gaussian, refresh = 0)

# 
# 
# 

### ```fit()```
### ```fit_xy()``` : 설명변수와 반응변수를 선택하여 적합

In [27]:
lm_model <- 
    linear_reg() %>%
    set_engine("lm")

lm_form_fit <- 
    lm_model %>%
    fit(Sale_Price ~ Longitude + Latitude, data = ames_train)

lm_xy_fit <-
    lm_model %>%
    fit_xy(x = ames_train %>% select(Longitude, Latitude),
           y = ames_train %>% pull(Sale_Price))

In [31]:
lm_form_fit
lm_xy_fit

parsnip model object


Call:
stats::lm(formula = Sale_Price ~ Longitude + Latitude, data = data)

Coefficients:
(Intercept)    Longitude     Latitude  
 -130898285      -804345      1326466  


parsnip model object


Call:
stats::lm(formula = ..y ~ ., data = data)

Coefficients:
(Intercept)    Longitude     Latitude  
 -130898285      -804345      1326466  


# 
# 
#### 랜덤 포레스트 적합 시
| 인수 유현 | ranger 패키지 | randomForest 패키지 | sparklyr 패키지 | parsnip 패키지 | 
| -- | -- | -- | -- | -- |
| # 사용할 변수 개수 | ```mtry``` | ```mtry``` | ```feature_subset_strategy``` | ```mtry``` |
| # 생성할 tree수 | ```num.trees``` | ```ntree``` | ```num_trees``` | ```trees``` |
| # 최소 가지 크기 | ```min.node.size``` | ```nodesize``` | ```min_instances_per_node``` | ```min_n``` | 

In [34]:
rand_forest(trees = 1000, min_n = 5) %>% 
    set_engine("ranger") %>% 
    set_mode("regression") %>% 
    translate()

Random Forest Model Specification (regression)

Main Arguments:
  trees = 1000
  min_n = 5

Computational engine: ranger 

Model fit template:
ranger::ranger(x = missing_arg(), y = missing_arg(), weights = missing_arg(), 
    num.trees = 1000, min.node.size = min_rows(~5, x), num.threads = 1, 
    verbose = FALSE, seed = sample.int(10^5, 1))

In [36]:
rand_forest(trees = 1000, min_n = 5) %>%
    set_engine("ranger", verbose = TRUE) %>%
    set_mode("regression")

Random Forest Model Specification (regression)

Main Arguments:
  trees = 1000
  min_n = 5

Engine-Specific Arguments:
  verbose = TRUE

Computational engine: ranger 


# 
# 
## 모델 결과

### ```extract_fit_engine()``` : 모델 출력 결과 활용

In [39]:
lm_form_fit %>% extract_fit_engine()


Call:
stats::lm(formula = Sale_Price ~ Longitude + Latitude, data = data)

Coefficients:
(Intercept)    Longitude     Latitude  
 -130898285      -804345      1326466  


In [40]:
lm_form_fit %>% extract_fit_engine() %>% vcov()

,(Intercept),Longitude,Latitude
(Intercept),4.338180e+13,327989572678,-301367952873
Longitude,3.279896e+11,3413902123,-197513889
Latitude,-3.013680e+11,-197513889,6729467557


In [43]:
(lm_form_fit %>%
    extract_fit_engine() %>% summary() -> model_res)


Call:
stats::lm(formula = Sale_Price ~ Longitude + Latitude, data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-143925  -46076  -18906   32309  407776 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -130898285    6586486  -19.87   <2e-16 ***
Longitude      -804345      58429  -13.77   <2e-16 ***
Latitude       1326466      82033   16.17   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 72430 on 2339 degrees of freedom
Multiple R-squared:  0.1563,	Adjusted R-squared:  0.1556 
F-statistic: 216.7 on 2 and 2339 DF,  p-value: < 2.2e-16


In [46]:
param_est <- coef(model_res)
class(param_est) ; param_est

[1] "matrix" "array"

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),-130898285.1,6586486.12,-19.87377,2.441170e-81
Longitude,-804345.1,58428.61,-13.76629,1.615072e-41
Latitude,1326465.5,82033.33,16.16984,7.825498e-56


In [47]:
tidy(lm_form_fit)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-130898285.1,6586486.12,-19.87377,2.441170e-81
Longitude,-804345.1,58428.61,-13.76629,1.615072e-41
Latitude,1326465.5,82033.33,16.16984,7.825498e-56


# 
# 
## 에측

### Tidymodels의 ```predict``` 사용시 ```type```의 종류
| 유형 | 반환 열 | 
| -- | -- |
| numeric | .pred | 
| class | .pred_class | 
| prob | .pred_{class levels} | 
| conf_int | .pred_lower / .pred_upper | 
| pred_int | .pred_lower / .pred_upper | 

In [48]:
ames_test_small <- ames_test %>% slice(1:5)

In [50]:
predict(lm_form_fit, new_data =  = ames_test_small)

.pred
<dbl>
210691.6
193732.3
190665.7
190678.5
217014.7


In [51]:
ames_test_small %>%
    select(Sale_Price) %>%
    bind_cols(predict(lm_form_fit, ames_test_small)) %>%
    bind_cols(predict(lm_form_fit, ames_test_small, type = "pred_int"))

Sale_Price,.pred,.pred_lower,.pred_upper
<int>,<dbl>,<dbl>,<dbl>
213500,210691.6,68546.79,352836.4
149900,193732.3,51616.01,335848.7
96000,190665.7,48559.32,332772.0
105500,190678.5,48572.97,332784.1
205000,217014.7,74891.52,359138.0


# 
# 
#### glmnet 예측 유형에 대한 다양한 반환 값
| 예측 유형 | 반환 값 | 
| -- | -- |
| numeric | numeric 행렬 |
| class | character 행렬 |
| probability (이진분류) |  numeric 행렬 | 
| probability (다지분류) | numeric 배열 |

# 
# 
# 
## 모델 사양 작성
- parsnip과 연동 가능하여 생성한 모든 모델 확인

In [ ]:
parsnip_addin()


Listening on http://127.0.0.1:6192



# 
# 
### tallmodels 패키지 
- parsnip 모델 정의

# 
### discrim 패키지
- 판별 분석 방법에 대한 모델 정의

# 

### parsnip과 함께 사용할 수 있는 모델 목록
https://www.tidymodels.org/find/